In [12]:
import pandas as pd

In [13]:
data = pd.read_csv('../data/disambiguationBenchmarkLabels.csv')
#data = pd.read_csv('/gpfs/sciencegenome/WoS-disambiguation/validation/validation-disambiguated-authors.csv')

In [14]:
data.columns

Index(['WoSid', 'MAGids', 'authorids', 'authorOrders', 'affiliationids',
       'affiliationNormalized', 'affiliationNames', 'GRIDids', 'SSdois',
       'SSauthorNameList', 'SSauthorIDList', 'SSrefList', 'ID_researcher',
       'Author_name', 'author_order'],
      dtype='object')

In [15]:
data = data.rename(columns = {"ID_researcher":"true_author_id", "authorids":"predicted_author_id"})


In [19]:
data[["true_author_id", "predicted_author_id"]].dropna()

,true_author_id,predicted_author_id
0,839,2900976016;2901401493
1,23852,1959309044;2103446411
2,5371,NaN
3,1475;2181;4643;9266;17626;28441,2124548152
4,11094,NaN
...,...,...
618492,24509,NaN
618493,6717,NaN
618494,15697,NaN
618495,29170,NaN


In [16]:
true_author_ids_group_by_predicted_result = {id_: list(rows['true_author_id']) for id_, rows in data.groupby('predicted_author_id')}
predicted_result_group_by_true_author = {id_: list(rows['predicted_author_id']) for id_, rows in data.groupby('true_author_id')}

In [17]:
from collections import Counter
def calculate_pairwise_correct_and_false_pair(author_group_dict):
    TP = 0
    FP = 0
    for id_, row in author_group_dict.items():
        length = len(row)
        counter_dict = Counter(row)
        temp_TP = 0
        for k,v in counter_dict.items():
            if v > 1:
                temp_TP += v * (v - 1)/ 2
        temp_FP = length * (length - 1) / 2 - temp_TP

        TP += temp_TP
        FP += temp_FP
        
    return TP, FP

def calculate_pairwise_F_measure(true_author_ids_group_by_predicted_result, predicted_result_group_by_true_author):
    TP, FP = calculate_pairwise_correct_and_false_pair(true_author_ids_group_by_predicted_result)
    _, FN = calculate_pairwise_correct_and_false_pair(predicted_result_group_by_true_author)
    
    pairwise_precision = TP / (TP + FP)
    pairwise_recall = TP / (TP + FN)
    pairwise_F = (2 * pairwise_precision * pairwise_recall)/(pairwise_precision + pairwise_recall)
    
    return pairwise_F
    


In [18]:
calculate_pairwise_F_measure(true_author_ids_group_by_predicted_result, predicted_result_group_by_true_author)

0.023815418773105276

In [6]:
# Let's test with the perfect case

perfect_true_author_ids_group_by_predicted_result = {id_: list(rows['predicted_author_id']) for id_, rows in data.groupby('predicted_author_id')}
perfect_predicted_result_group_by_true_author = {id_: list(rows['true_author_id']) for id_, rows in data.groupby('true_author_id')}

In [7]:
calculate_pairwise_F_measure(perfect_true_author_ids_group_by_predicted_result, perfect_predicted_result_group_by_true_author)

1.0